<a href="https://colab.research.google.com/github/fatemafaria142/Investigating-SQL-Query-Answer-Generation-Using-Different-Large-Language-Models/blob/main/SQL_Query_Answer_Generation_using_using_Llama_2_7b_chat_hf_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━

#### **Dataset Link:** https://huggingface.co/datasets/b-mc2/sql-create-context

In [2]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("b-mc2/sql-create-context")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# Select 500 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(500))

# Select 200 rows for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(500, 700))


In [4]:
train_subset

Dataset({
    features: ['question', 'answer', 'context'],
    num_rows: 500
})

In [5]:
test_subset

Dataset({
    features: ['question', 'answer', 'context'],
    num_rows: 200
})

In [6]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 500
Test subset size: 200


In [7]:
# Show 5 examples from the train dataset
print("Train subset examples:")
for i in range(5):
    context = train_subset['context'][i]
    question = train_subset['question'][i]
    answer = train_subset['answer'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("-------------------")

Train subset examples:
Example 1:
Context: CREATE TABLE head (age INTEGER)
Question: How many heads of the departments are older than 56 ?
Answer: SELECT COUNT(*) FROM head WHERE age > 56
-------------------
Example 2:
Context: CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)
Question: List the name, born state and age of the heads of departments ordered by age.
Answer: SELECT name, born_state, age FROM head ORDER BY age
-------------------
Example 3:
Context: CREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR)
Question: List the creation year, name and budget of each department.
Answer: SELECT creation, name, budget_in_billions FROM department
-------------------
Example 4:
Context: CREATE TABLE department (budget_in_billions INTEGER)
Question: What are the maximum and minimum budget of the departments?
Answer: SELECT MAX(budget_in_billions), MIN(budget_in_billions) FROM department
-------------------
Example 5:
Context: CREATE TABLE d

In [8]:
# Show 5 examples from the test dataset
print("Test subset examples:")
for i in range(5):
    context = test_subset['context'][i]
    question = test_subset['question'][i]
    answer = test_subset['answer'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("-------------------")

Test subset examples:
Example 1:
Context: CREATE TABLE services (service_name VARCHAR, service_id VARCHAR); CREATE TABLE first_notification_of_loss (service_id VARCHAR)
Question: Find the name of services that have been used for more than 2 times in first notification of loss.
Answer: SELECT t2.service_name FROM first_notification_of_loss AS t1 JOIN services AS t2 ON t1.service_id = t2.service_id GROUP BY t1.service_id HAVING COUNT(*) > 2
-------------------
Example 2:
Context: CREATE TABLE settlements (claim_id VARCHAR, settlement_amount INTEGER); CREATE TABLE claims (Effective_Date VARCHAR, claim_id VARCHAR)
Question: What is the effective date of the claim that has the largest amount of total settlement?
Answer: SELECT t1.Effective_Date FROM claims AS t1 JOIN settlements AS t2 ON t1.claim_id = t2.claim_id GROUP BY t1.claim_id ORDER BY SUM(t2.settlement_amount) DESC LIMIT 1
-------------------
Example 3:
Context: CREATE TABLE customers (customer_id VARCHAR, customer_name VARCHAR); CR

#### **Creating a Mental Health Support Prompt Using LangChain**

In [11]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Creating prompt for large language model
    pre_prompt = """[INST] <<SYS>>\nYour task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.\n"""
    prompt = pre_prompt + "Here is the table I have created: {context}" + " and My question is about the table is: {question}"+ "[\INST]"+"\nAnswer: {answer}"

    # Extract context, question, and answer from the sample
    example_context = sample['context']
    example_question = sample['question']
    example_answer = sample['answer']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=["context", "question","answer"])
    single_turn_prompt = prompt_template.format(context=example_context, question=example_question, answer=example_answer)

    return single_turn_prompt


#### **Generating LangChain Prompt for a Training Subset Examples**

In [12]:
prompt = create_prompt(train_subset[0])
print(prompt)

[INST] <<SYS>>
Your task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.
Here is the table I have created: CREATE TABLE head (age INTEGER) and My question is about the table is: How many heads of the departments are older than 56 ?[\INST]
Answer: SELECT COUNT(*) FROM head WHERE age > 56


In [13]:
prompt = create_prompt(train_subset[1])
print(prompt)

[INST] <<SYS>>
Your task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.
Here is the table I have created: CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR) and My question is about the table is: List the name, born state and age of the heads of departments ordered by age.[\INST]
Answer: SELECT name, born_state, age FROM head ORDER BY age


In [14]:
prompt = create_prompt(train_subset[2])
print(prompt)

[INST] <<SYS>>
Your task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.
Here is the table I have created: CREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR) and My question is about the table is: List the creation year, name and budget of each department.[\INST]
Answer: SELECT creation, name, budget_in_billions FROM department


#### **Generating LangChain Prompt for a Testing Subset Examples**

In [15]:
prompt = create_prompt(test_subset[0])
print(prompt)

[INST] <<SYS>>
Your task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.
Here is the table I have created: CREATE TABLE services (service_name VARCHAR, service_id VARCHAR); CREATE TABLE first_notification_of_loss (service_id VARCHAR) and My question is about the table is: Find the name of services that have been used for more than 2 times in first notification of loss.[\INST]
Answer: SELECT t2.service_name FROM first_notification_of_loss AS t1 JOIN services AS t2 ON t1.service_id = t2.service_id GROUP BY t1.service_id HAVING COUNT(*) > 2


In [16]:
prompt = create_prompt(test_subset[1])
print(prompt)

[INST] <<SYS>>
Your task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.
Here is the table I have created: CREATE TABLE settlements (claim_id VARCHAR, settlement_amount INTEGER); CREATE TABLE claims (Effective_Date VARCHAR, claim_id VARCHAR) and My question is about the table is: What is the effective date of the claim that has the largest amount of total settlement?[\INST]
Answer: SELECT t1.Effective_Date FROM claims AS t1 JOIN settlements AS t2 ON t1.claim_id = t2.claim_id GROUP BY t1.claim_id ORDER BY SUM(t2.settlement_amount) DESC LIMIT 1


In [17]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST] <<SYS>>
Your task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.
Here is the table I have created: CREATE TABLE customers (customer_id VARCHAR, customer_name VARCHAR); CREATE TABLE customers_policies (customer_id VARCHAR) and My question is about the table is: How many policies are listed for the customer named "Dayana Robel"?[\INST]
Answer: SELECT COUNT(*) FROM customers AS t1 JOIN customers_policies AS t2 ON t1.customer_id = t2.customer_id WHERE t1.customer_name = "Dayana Robel"


In [18]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST] <<SYS>>
Your task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.
Here is the table I have created: CREATE TABLE customers (customer_id VARCHAR, customer_name VARCHAR); CREATE TABLE customers_policies (customer_id VARCHAR) and My question is about the table is: How many policies are listed for the customer named "Dayana Robel"?[\INST]
Answer: SELECT COUNT(*) FROM customers AS t1 JOIN customers_policies AS t2 ON t1.customer_id = t2.customer_id WHERE t1.customer_name = "Dayana Robel"


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [20]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

In [23]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [24]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [25]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [27]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.8  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300, do_sample=do_sample, pad_token_id=pad_token_id,  top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)


# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nYour task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.\n"""
prompt = pre_prompt + "Here is the table I have created: {context}" + " and My question is about the table is: {question}"+ "[\INST]"

# Create a prompt using the template and the example
prompt= PromptTemplate(template=prompt, input_variables=["context", "question"])


chain = prompt | llm

context = '''CREATE TABLE customers (customer_id VARCHAR, customer_name VARCHAR);
             CREATE TABLE customers_policies (customer_id VARCHAR)'''
question = '''How many policies are listed for the customer named "Dayana Robel"?'''

print(chain.invoke({"context": context, "question":question}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Great, let's get started! To retrieve the number of policies listed for a specific customer named "Dayana Robel", you can use the following SQL query:

SELECT COUNT(*) FROM customers_policies WHERE customer_id = 'Dayana Robel';

Explanation:

* `SELECT`: This keyword is used to select data from a table.
* `COUNT(*)`: This function counts the number of rows in a table. In this case, we want to count the number of policies listed for a specific customer.
* `FROM`: This keyword is used to specify the table(s) from which data should be retrieved. In this case, we are retrieving data from the `customers_policies` table.
* `WHERE`: This keyword is used to specify conditions for which data should be retrieved. In this case, we are filtering the data to only include policies listed for the customer named "Dayana Robel".

Please let me know if you have any questions or if you'd like to try a different query!


### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [28]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [29]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [30]:
from transformers import TrainingArguments
output_model= "Llama_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [31]:
from trl import SFTTrainer

max_seq_length = 300

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [32]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.899900
20,1.163100
30,0.856600
40,0.668600
50,0.585000
60,0.510800
70,0.496200
80,0.467300
90,0.462600
100,0.486000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=150, training_loss=0.6463010390599568, metrics={'train_runtime': 904.4084, 'train_samples_per_second': 0.663, 'train_steps_per_second': 0.166, 'total_flos': 7140460953600000.0, 'train_loss': 0.6463010390599568, 'epoch': 2.84})

### **Save the model**

In [33]:
trainer.save_model("Llama_instruct_generation")

In [34]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


### **Example: 01**

In [45]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nYour task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.\n"""
prompt = pre_prompt + "Here is the table I have created: {context}" + " and My question is about the table is: {question}"+ "[\INST]"

# Create a prompt using the template and the example
prompt= PromptTemplate(template=prompt, input_variables=["context", "question"])


chain = prompt | llm

context = '''CREATE TABLE loan (branch_id VARCHAR);
             CREATE TABLE bank (bname VARCHAR, branch_id VARCHAR)'''
question = '''Find the total amount of loans offered by each bank branch.'''

print(chain.invoke({"context": context, "question":question}))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


  Sure, I can help you with that! To find the total amount of loans offered by each bank branch, you can use a SQL query like this:

SELECT branch_id, SUM(loan_amount) AS total_loans
FROM loan
JOIN bank ON loan.branch_id = bank.branch_id
GROUP BY branch_id

This query joins the loan and bank tables on the branch_id column, then groups the results by the branch_id column to give you the total amount of loans offered by each branch.

Here's a breakdown of the query:

* SELECT: We select the branch_id and the SUM of the loan_amount column.
* FROM: We join the loan and bank tables on the branch_id column.
* JOIN: We join the loan and bank tables on the branch_id column.
* GROUP BY: We group the results by the branch_id column.

Let me know if you have any questions or if you'd like to see the query in action!


### **Example: 02**

In [46]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nYour task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.\n"""
prompt = pre_prompt + "Here is the table I have created: {context}" + " and My question is about the table is: {question}"+ "[\INST]"

# Create a prompt using the template and the example
prompt= PromptTemplate(template=prompt, input_variables=["context", "question"])


chain = prompt | llm

context = '''CREATE TABLE player (Player_name VARCHAR, Player_ID VARCHAR);
             CREATE TABLE coach (coach_name VARCHAR, Coach_ID VARCHAR);
             CREATE TABLE player_coach (Coach_ID VARCHAR, Player_ID VARCHAR)'''

question = '''Show the names of players and names of their coaches.'''

print(chain.invoke({"context": context, "question":question}))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


  Sure! Here are two SQL queries that you can use to retrieve the names of players and names of their coaches from the `player_coach` table:

Query 1: To retrieve the names of players
```
SELECT Player_name FROM player_coach;
```
This query will return all the `Player_name` values from the `player_coach` table.

Query 2: To retrieve the names of coaches
```
SELECT Coach_name FROM player_coach;
```
This query will return all the `Coach_name` values from the `player_coach` table.

By combining these two queries, you can retrieve the names of both players and coaches:
```
SELECT Player_name FROM player_coach UNION SELECT Coach_name FROM player_coach;
```
This query will return a single result set with both `Player_name` and `Coach_name` values.

Let me know if you have any questions or need further assistance!


### **Example: 03**

In [49]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nYour task is to write SQL queries to retrieve specific information from the given database. Please follow the instructions carefully and use the appropriate SQL syntax.\n"""
prompt = pre_prompt + "Here is the table I have created: {context}" + " and My question is about the table is: {question}"+ "[\INST]"

# Create a prompt using the template and the example
prompt= PromptTemplate(template=prompt, input_variables=["context", "question"])

chain = prompt | llm

context = '''CREATE TABLE university (school_id VARCHAR);
             CREATE TABLE basketball_match (school_id VARCHAR)'''

question = '''How many schools do not participate in the basketball match?'''

print(chain.invoke({"context": context, "question":question}))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


  To retrieve the number of schools that do not participate in the basketball match, you can use the following SQL query:
```
SELECT COUNT(*)
FROM university
WHERE NOT EXISTS (SELECT 1
                  FROM basketball_match
                  WHERE university.school_id = basketball_match.school_id);
```
Explanation:

* `SELECT COUNT(*)` retrieves the total number of rows in the table.
* `FROM university` specifies the table to query.
* `WHERE NOT EXISTS` is a subquery that retrieves the number of rows in the `basketball_match` table where the `school_id` column matches the `school_id` column in the `university` table.
* `NOT EXISTS` is the opposite of `EXISTS`, which retrieves the number of rows in the `basketball_match` table where the `school_id` column matches the `school_id` column in the `university` table.

This query will return the number of schools that do not participate in the basketball match.
